In [44]:
# name of student org, website link (if there are any), instagram link (if there are any)
from requests import get
from bs4 import BeautifulSoup
import csv

data = []

list = ["Academic - Engineering","Academic - Agriculture and Life Sciences","Academic - Architecture",
        "Academic - Government and Public Service","Academic - Education and Human Development",
        "Academic - Engineering","Academic - Geosciences","Academic - Health Sciences",
        "Academic - Honors","Academic - Law","Academic - Liberal Arts","Academic - Business",
        "Academic - Science","Academic - Veterinary Medicine and Biomedical Sciences",
        "Arts/ Entertainment","Fraternity/Sorority Life","International/Multicultural",
        "Leadership/Governance","Military","Politics/Advocacy","Professional/Career","Recreation/Sports",
        "Religious/Spiritual","Residence Halls","Service","Social","Special Interest"]
for i in list:
    print(f"Searching for organizations [{i}]", end="")
    converted = i.replace(" ","%20")
    url = f"https://stuactonline.tamu.edu/app/search/index/index/search/category?q={converted}"
    
    response = get(url).text
    soup = BeautifulSoup(response, "html.parser")
    table = soup.find('table',{"class":"table table-striped"})
    blocks = table.find_all('tr')
    for j in blocks:
        check = j.find_all('td')[1]
        check2 = check.find('small').text

        if "Pending Recognition" in check2.strip():
            continue
        elif "Not Recognized" in check2.strip() or "Exempt from Recognition" in check2.strip():
            break
        link = j.find('a')
        soup2 = BeautifulSoup(get(link["href"]).text, "html.parser")
        name = soup2.find('h1', {'class':'title'}).text
        
        org = {
            "name":name,
            "email":None,
            "url":None
        }
        try:
            group = soup2.find_all('dl')[2]
        except IndexError:
            continue
        urls = group.find_all('a')
        for k in urls:
            if is_email(k['href']):
                org["email"] = k['href'][7:]
            elif is_url(k['href']):
                org["url"] = k['href']
        data.append(org)
        print(f"{org}", end="\n")
    print("")

keys = data[0].keys()

with open('organizations.csv', 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

print("Data successfully saved to organizations.csv")

Searching for organizations [Academic - Engineering]{'name': '12th Astronaut Lab', 'email': None, 'url': None}
{'name': '2D Visual Developers', 'email': '2dvisdev@gmail.com', 'url': 'https://linktr.ee/2dvd'}
{'name': 'Aerospace Engineering Graduate Student Association ', 'email': 'aegsa@tamu.edu', 'url': 'https://aegsa6.wixsite.com/aegsa'}
{'name': 'Aggie Aerospace Women in Engineering', 'email': 'aggieawe@gmail.com', 'url': 'http://AAWE.engr.tamu.edu'}
{'name': 'Aggie Club of Engineers', 'email': 'Tamuacepresident@gmail.com', 'url': 'http://aggieclubofengineers.org/'}
{'name': 'Aggie Competitive Programming Club', 'email': 'contact@tamuacpc.com', 'url': 'http://tamuacpc.com'}
{'name': 'Aggie Data Science Club', 'email': 'aggiedatascience@gmail.com', 'url': 'https://www.aggiedatascience.com/'}
{'name': 'Aggie Robotics', 'email': 'aggie.robotics@tamu.edu', 'url': 'https://www.aggierobotics.org/'}
{'name': 'Aggie Students Supporting Israel', 'email': 'ssihowdy@gmail.com', 'url': None}
{'

In [27]:
def is_email(s):
    if s.startswith("mailto:"):
        return True
    return False

In [26]:
def is_url(s):
    if s.startswith("http://") or s.startswith("https://"):
        return True
    return False